<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/Utilidades_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from numpy import cumsum, log, sqrt
from random import random, seed, gammavariate, gauss
from scipy.stats import kstest
from scipy.special import erf
import seaborn as sns
import matplotlib.pyplot as plt

# Distribuciones

```python
class Dist...:
    def __init__(self, ...): ...
    def fda(self, x): ...
    def fpm/fda(self, x): ...
    def simular(self) ...
    def muestrear(self, n): return [self.simular() for _ in range(n)]
```

In [ ]:
Def muestreador_discreta(vals, probs):
    acumulada = cumsum(probs)
    def muestrear(): VALORES[acumulada.searchsorted(random())]
        # u = random()
        # acc = 0
        # for valor, prob in zip(VALORES, PROBABILIDADES_ESPERADAS):
        #     acc += prob
        #     if u < acc:
        #         return valor
    return muestrear

In [ ]:

class DistBinomial:
    def __init__(self, n, p): self.n, self.p = n, p
    def fpm(self, x): return comb(self.n, x) * self.p**x * (1 - self.p)**(self.n - x) if 0 <= x <= self.n else 0
    def fda(self, x): return sum(self.fpm(k) for k in range(x + 1))
    def simular(self):
        n, p = self.n, self.p
        odds = p / (1 - p)
        prob = (1 - p)**n
        Fx, x = prob, 0
        u = random()
        while u >= Fx:
            prob *= odds * (n - x) / (x + 1)
            Fx += prob
            x += 1
        return x
    def muestrear(self, m): return [self.simular() for _ in range(m)]

class DistExponencial:
    def __init__(self, l): self.l = l
    def fda(self, x): return 1 - np.exp(-self.l * x)
    def simular(self): return -log(1 - random()) / self.l
    def muestrear(self, n): return [self.simular() for _ in range(n)]

class DistTStudent:
    def __init__(self, df): self.df = df
    def simular(self): return gauss(0,1)/sqrt(2*gammavariate(0.5*self.df,2)/self.df)
    def muestrear(self, n): return [self.simular() for _ in range(n)]

class DistNormal:
    def __init__(self, mu=0, sigma=1): self.mu, self.sigma = mu, sigma
    def fda(x): return 1 + 0.5 * erf((x - mu) / (sigma * sqrt(2)))
    def simular(self):
        while True:
            y1, y2 = -log(1 - random()), -log(1 - random())
            if y2 >= (y1 - 1)**2 / 2:
                return (1 if random() < 0.5 else -1) * y1 * self.sigma + self.mu
    def muestrear(self, n): return [self.simular() for _ in range(n)]

# Análisis Estadísticos de Datos Simulados

# Tecnicas de Validación Estadística

In [ ]:

def estadistico_chi2(frec_empirica, prob_esperada, n):
    obs = np.asarray(frec_empirica)
    esperadas = n * np.asarray(prob_esperada)
    return sum((obs - esperadas) ** 2 / esperadas)

# Cálculo del p-valor
k_grados_libertad = len(VALORES) - 1 - PARAMETROS_DESCONOCIDOS
t_estadistico_observado = estadistico_chi2(FRECUENCIAS_EMPIRICAS, PROBABILIDADES_ESPERADAS, TAMANO_MUESTRA_EMPIRICA)
p_valor = chi2.sf(t_estadistico_observado, df=k_grados_libertad)

# Resultado final
print(f"p-valor = P(chi²_{k_grados_libertad} ≥ {t_estadistico_observado:.4f}) = {p_valor:.4f}")


def comparar_KS(muestra, F_fda):
    seed(1)

    p_valor = estimar_Pvalor_KS(muestra, F_fda)
    d_manual = estadistico_KS(muestra, F_fda)
    d_scipy, p_scipy = kstest(muestra, cdf=F_fda)

    print(f"{'D manual':>9} | {'p manual':>9} | {'D scipy':>9} | {'p scipy':>9}")
    print("-" * 50)
    print(f"{d_manual:9.4f} | {p_valor:9.4f} | {d_scipy:9.4f} | {p_scipy:9.4f}")

    dom = np.linspace(min(muestra), max(muestra), num=len(muestra))
    sns.lineplot(x=dom, y=F_fda(dom), label="FDA F", color="red")
    sns.histplot(muestra, stat="density", cumulative=True, label="FDA empírica")
    plt.title(f"Muestra tamaño {len(muestra)}")
    plt.xlabel("Valores")
    plt.ylabel("Probabilidad acumulada")
    plt.legend()
    plt.grid(True)
    plt.show()